In [21]:
import keras
keras.__version__

'2.13.1'

In [10]:
from keras.preprocessing.image import load_img, img_to_array

# Calea catre imaginea pe care vrem sa o transformam
target_image_path = 'C:/Users/danij/Desktop/Practica/Carte/Imagini_de_combinat/casa.jpg'
# Calea catre imaginia de stil
style_reference_image_path = 'C:/Users/danij/Desktop/Practica/Carte/Imagini_de_combinat/style.jpg'

# Dimensiunile imaginii generate
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

In [11]:
# Vom avea nevoie de niste functii auxiliare pentru incarcare, preprocesare si post procesare a imaginilor
import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    # Eliminarea centru-zero prin valoarea medie a pixelilor
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [12]:
from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))

# In acest marcator vom avea imaginea generata
combination_image = K.placeholder((1, img_height, img_width, 3))

# Combinam cele 3 imagini intr-un singur batch
input_tensor = K.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)

# Construim networkul VGG19 network cu batch-ul nostru de 3 imagini ca si input 
# Modelul va porni cu greutatile ImageNet preantrenate 
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)
print('Model loaded.')

Model loaded.


In [13]:
# Definim pierderile
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [14]:
# Aici avem pierderile stilului
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram


def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [15]:
# La cele 2 componente de pierderi adaugam o a treia numita "variatia totala a pierderii", ajuta la regularizarea pierderilor
def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [16]:
# Dictionarul care face maparea numelor straturilor la tensorii de activare
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
# Numele stratului folosit pentru continutul pierdut
content_layer = 'block5_conv2'
# Numele straturilor folosite pentru stilul pierdut
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
# Greutatea medie a componentelor pierdute
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

# Definim pierderile adaugand toate componentele la o "variabila de pierderi"
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(target_image_features,
                                      combination_features)
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(style_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [27]:
import keras.backend as K
import tensorflow as tf

# Preluam gradientii pierderilor imaginii generate
with tf.GradientTape() as tape:
    grads = K.GradientTape(loss, model.input)[0]

# Functia pentru a recupera valorile gradientilor si pierderilor curente
fetch_loss_and_grads = K.function([combination_image], [loss, grads])


class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

AttributeError: module 'keras.backend' has no attribute 'GradientTape'

In [ ]:
# In final continuam procesul de "ascensiune a gradientului" salvand fiecare imagine curenta generata la fiecare iteratie
# a algoritmului
# Aici avem o iteratie ce reprezinta 20 de pasi de ascenciune a gradientului
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

result_prefix = 'style_transfer_result'
iterations = 20

# Minimizam pierderile neurale a stilului
# Starea initiala: imaginea tinta

x = preprocess_image(target_image_path)
x = x.flatten()
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # Salvam curenta imagine generata
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

In [ ]:
from matplotlib import pyplot as plt

# Imaginea continut
plt.imshow(load_img(target_image_path, target_size=(img_height, img_width)))
plt.figure()

# Imaginea stil
plt.imshow(load_img(style_reference_image_path, target_size=(img_height, img_width)))
plt.figure()

# imaginea generata
plt.imshow(img)
plt.show()